In [1]:
import json
import re
from collections import Counter, defaultdict
from typing import Any, Dict, List

from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import eli5

Array = Any

In [2]:
filename = "dataset/clean_csv/text.csv"

df = pd.read_csv(filename)

In [3]:
df = df[['date', 'clipTarget', 'description']]
df_train = df.iloc[:324160, :]
df_train = df_train.dropna(subset=['description']).reset_index(drop=True)


df_test = df.iloc[324160:, :]
df_test = df_test.dropna(subset=['description']).reset_index(drop=True)
del df

In [47]:
def text_preprocessing(data):
    stop = stopwords.words('russian')
    data  = " ".join(x.lower() for x in data.split())
    data = data.replace('[^\w\s]','')
    data = data.replace('#',' ')
    data = " ".join(x for x in data.split() if x not in stop)
    return data

In [48]:
text_preprocessing("#массажлицакурган#массажкурган#курган")

'массажлицакурган массажкурган курган'

In [49]:
def simple_preprocessing(data):
    stop = stopwords.words('russian')
    data = data.apply(lambda x: " ".join(x.lower() for x in x.split()))
    data = data.str.replace('[^\w\s]','')
    data = data.str.replace('#',' ')
    data = data.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    return data

In [50]:
df_train['clean_text'] = df_train["description"].apply(lambda row: text_preprocessing(row))

In [51]:
df_train.sample(5)

,date,clipTarget,description,clean_text
5992,2022-03-26,CREATION,"Ну что, привет понедельник Привет новая жизнь ...","что, привет понедельник привет новая жизнь инс..."
120726,2022-05-08,HEALTH,#токсичныеотношения #осознанность #осознанност...,токсичныеотношения осознанность осознанностьжи...
180141,2022-06-08,DANCE,Совсем забыла выставить самую первую хореограф...,забыла выставить самую первую хореографию деву...
47876,2022-04-11,ANIMALS,#Istanbul #turkey #galatasaray,istanbul turkey galatasaray
84948,2022-04-26,FAMILY,С днём рождения❤️,днём рождения❤️


In [52]:
df_test['clean_text'] = df_test["description"].apply(lambda row: text_preprocessing(row))

In [53]:
df_test.sample(5)

,date,clipTarget,description,clean_text
19534,2022-07-07,BEAUTY,когда хочется чего-то яркого и в тоже время не...,хочется чего-то яркого время нежного - выбирай...
38055,2022-07-20,FOOD,Медовик домашний 🍯 лёгкий мазохизм. Ну а что? ...,медовик домашний 🍯 лёгкий мазохизм. что? дальш...
4981,2022-07-07,FAMILY,Армянская свадьба🤵‍♂️👰‍♀️ #армения #ереван #ар...,армянская свадьба🤵‍♂️👰‍♀️ армения ереван армян...
26116,2022-07-07,MOVIES,Eren Yeager #эрен_йегер #anime #edit #эдитаним...,eren yeager эрен_йегер anime edit эдитаниме ry...
5339,2022-07-07,MUSIC,big baby tape x problems #битмейкер #битмейкинг,big baby tape x problems битмейкер битмейкинг


In [54]:
log_reg = LogisticRegression(solver='newton-cg')
vectorizer = CountVectorizer()

In [55]:
X_train_text = vectorizer.fit_transform(df_train['clean_text'].to_list())
X_test_text = vectorizer.transform(df_test['clean_text'].to_list())

In [56]:
pipe = Pipeline([('clf', log_reg)])

In [57]:
%%time
pipe.fit(X_train_text, df_train["clipTarget"].to_list())

CPU times: user 20min 37s, sys: 44min 58s, total: 1h 5min 35s
Wall time: 12min 21s


Pipeline(steps=[('clf', LogisticRegression(solver='newton-cg'))])

In [58]:
print(roc_auc_score(df_test["clipTarget"].to_list(), pipe.predict_proba(X_test_text), multi_class='ovr'))
print(f1_score(df_test["clipTarget"].to_list(), pipe.predict(X_test_text), average='macro'))

0.8380153811339905
0.3194419692340488


In [59]:
eli5.show_weights(estimator=pipe, feature_names= list(vectorizer.get_feature_names()), top=(50, 5))

/home/xrenya/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [60]:
df = pd.read_csv(filename)
df = df[['date', 'clipTarget', 'asr']]
df_train = df.iloc[:324160, :]
df_train = df_train.dropna(subset=['asr']).reset_index(drop=True)


df_test = df.iloc[324160:, :]
df_test = df_test.dropna(subset=['asr']).reset_index(drop=True)
del df

In [61]:
df_train.sample(5)

,date,clipTarget,asr
4904,2022-04-07,MUSIC,А мне бы просто один парадный не были бы тольк...
4246,2022-04-06,MOVIES,Красивая речь. Вообще что это? Давайте разбере...
5827,2022-04-07,HEALTH,"Так, а что такое с пациенты, что есть, что и к..."
2134,2022-04-06,CREATION,А моего тела объединение этого мира единения м...
3919,2022-04-06,FOOD,Салатик готовится очень легко для салата. Нам ...


In [62]:
df_train['clean_text'] = df_train["asr"].apply(lambda row: text_preprocessing(row))

In [63]:
df_train.sample(5)

,date,clipTarget,asr,clean_text
2953,2022-04-06,NATURE,В жизни обязательно должны быть паузы такие па...,"жизни обязательно должны паузы такие паузы, ва..."
4550,2022-04-06,SPORT,"Мы сыграли в футбол, моя команда. Идите сюда, ...","сыграли футбол, команда. идите сюда, победа че..."
6171,2022-04-07,FITNESS,У тебя все отлично получается. Не сдавайся. Пр...,отлично получается. сдавайся. просто делай мал...
4679,2022-04-06,POSING,"Моя старая полар Нана, моя тора полови нара на...","старая полар нана, тора полови нара бутыл кирина."
1709,2022-04-06,DRAWING,"Быть художником, не просто ведь каждый художни...","художником, просто каждый художник, проснувшис..."


In [64]:
df_test['clean_text'] = df_test["asr"].apply(lambda row: text_preprocessing(row))

In [65]:
log_reg = LogisticRegression(solver='newton-cg')
vectorizer = CountVectorizer()

In [66]:
X_train_text = vectorizer.fit_transform(df_train['clean_text'].to_list())
X_test_text = vectorizer.transform(df_test['clean_text'].to_list())

In [67]:
pipe = Pipeline([('clf', log_reg)])

In [68]:
%%time
pipe.fit(X_train_text, df_train["clipTarget"].to_list())

CPU times: user 1min 19s, sys: 3min 43s, total: 5min 3s
Wall time: 19.8 s


Pipeline(steps=[('clf', LogisticRegression(solver='newton-cg'))])

In [69]:
# print(roc_auc_score(df_test["clipTarget"].to_list(), pipe.predict_proba(X_test_text), multi_class='ovr'))
print(f1_score(df_test["clipTarget"].to_list(), pipe.predict(X_test_text), average='macro'))

0.09779490223667905


In [70]:
y_preds_test = pipe.predict_proba(X_test_text)
y_preds_train = pipe.predict_proba(X_train_text)
y_true = df_test["clipTarget"].to_list() + df_train["clipTarget"].to_list()
print(roc_auc_score(y_true, np.concatenate((y_preds_test, y_preds_train)), multi_class='ovr'))

0.983435740054089


In [71]:
eli5.show_weights(estimator=pipe, feature_names= list(vectorizer.get_feature_names()), top=(50, 5))

/home/xrenya/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
df = pd.read_csv(filename)
df_train = df.iloc[:324160, :]
df_train = df_train.reset_index(drop=True)


df_test = df.iloc[324160:, :]
df_test = df_test.reset_index(drop=True)
del df

In [18]:
df_train["asr"] = df_train["asr"].apply(lambda row: row if isinstance(row, str) else "")
df_train["description"] = df_train["description"].apply(lambda row: row if isinstance(row, str) else "")
df_train["text"] = df_train["asr"] + df_train["description"]

In [19]:
columns = ["clipTarget", "text"]

In [20]:
df_train = df_train[df_train["text"] != ""].reset_index(drop=True)
df_train = df_train[columns].reset_index(drop=True)
df_train

,clipTarget,text
0,ANIMALS,🐶🐕🐩🐾 #ЦвергПинчер #МиниДоберман #ФедяПинчер #F...
1,DANCE,🤩Подруга-ночь🌃-мне не до сна 🙅‍♀️💃👏 #подругано...
2,FOOD,"Всем, привет Я обожаю делать приятные сюрпризы..."
3,ANIMALS,Коротко о нашем коте Лемуре😁#farm #farmer #far...
4,FITNESS,"Если не можешь, значит, можешь🤍 #ironcros#gym#..."
...,...,...
229077,CULTURE,#высотанавека #высота #верёвки #жумар #жумарь ...
229078,KIDS,Радость в мелочах 🍁 #счастьеесть #счастье #доч...
229079,SPORT,Танец сегодня – это способ самовыражения. Кто-...
229080,BEAUTY,"🙀У нас был всего час времени 🧚🏻‍♀️ , ты просто..."


In [21]:
df_train['clean_text'] = df_train["text"].apply(lambda row: text_preprocessing(row))
df_train

,clipTarget,text,clean_text
0,ANIMALS,🐶🐕🐩🐾 #ЦвергПинчер #МиниДоберман #ФедяПинчер #F...,🐶🐕🐩🐾 цвергпинчер минидоберман федяпинчер fedya...
1,DANCE,🤩Подруга-ночь🌃-мне не до сна 🙅‍♀️💃👏 #подругано...,🤩подруга-ночь🌃-мне сна 🙅‍♀️💃👏 подруганочь деву...
2,FOOD,"Всем, привет Я обожаю делать приятные сюрпризы...","всем, привет обожаю делать приятные сюрпризы л..."
3,ANIMALS,Коротко о нашем коте Лемуре😁#farm #farmer #far...,коротко нашем коте лемуре😁 farm farmer farmani...
4,FITNESS,"Если не можешь, значит, можешь🤍 #ironcros#gym#...","можешь, значит, можешь🤍 ironcros gym fitnes to..."
...,...,...,...
229077,CULTURE,#высотанавека #высота #верёвки #жумар #жумарь ...,высотанавека высота верёвки жумар жумарь сезон...
229078,KIDS,Радость в мелочах 🍁 #счастьеесть #счастье #доч...,радость мелочах 🍁 счастьеесть счастье доченька...
229079,SPORT,Танец сегодня – это способ самовыражения. Кто-...,танец сегодня – это способ самовыражения. кто-...
229080,BEAUTY,"🙀У нас был всего час времени 🧚🏻‍♀️ , ты просто...","🙀у час времени 🧚🏻‍♀️ , просто фея-волшебница ❤..."


In [22]:
df_test["asr"] = df_test["asr"].apply(lambda row: row if isinstance(row, str) else "")
df_test["description"] = df_test["description"].apply(lambda row: row if isinstance(row, str) else "")
df_test["text"] = df_test["asr"] + df_test["description"]

In [23]:
df_test = df_test[df_test["text"] != ""].reset_index(drop=True)
df_test = df_test[columns].reset_index(drop=True)
df_test

,clipTarget,text
0,DANCE,#ЯТвойКосмос #Гагарин
1,FAMILY,Первый раз на море 😍счастье наше❤️😘
2,FITNESS,Открытые занятия в усадьбе Кривякино каждое во...
3,ANIMALS,После умственного тренинга можно и на тарзанке...
4,FAMILY,"июня года в . , в очень жаркий петербургский д..."
...,...,...
38530,HUMOR,"Вчера снимали новый выпуск Шоперов, в пятницу ..."
38531,FITNESS,Если было полезно ставь ❤️ #тренировкидома #зд...
38532,FITNESS,Лицом по касательной к стене прошел😂✈️
38533,KIDS,❤️😋


In [24]:
df_test['clean_text'] = df_test["text"].apply(lambda row: text_preprocessing(row))

In [25]:
pipe = Pipeline([('clf', log_reg)])

In [26]:
log_reg = LogisticRegression(solver='newton-cg')
vectorizer = CountVectorizer()

In [27]:
X_train_text = vectorizer.fit_transform(df_train['clean_text'].to_list())
X_test_text = vectorizer.transform(df_test['clean_text'].to_list())

In [28]:
%%time
pipe.fit(X_train_text, df_train["clipTarget"].to_list())

CPU times: user 24min 8s, sys: 47min 46s, total: 1h 11min 54s
Wall time: 13min 59s


Pipeline(steps=[('clf', LogisticRegression(solver='newton-cg'))])

In [29]:
print(roc_auc_score(df_test["clipTarget"].to_list(), pipe.predict_proba(X_test_text), multi_class='ovr'))
print(f1_score(df_test["clipTarget"].to_list(), pipe.predict(X_test_text), average='macro'))

0.8372085920568483
0.31925652995414744


In [30]:
eli5.show_weights(estimator=pipe, feature_names= list(vectorizer.get_feature_names()), top=(50, 5))

/home/xrenya/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [31]:
df = pd.read_csv(filename)


df = df[['date', 'clipTarget', 'description']]
df_train = df.iloc[:324160, :]
df_train = df_train.dropna(subset=['description']).reset_index(drop=True)


df_test = df.iloc[324160:, :]
df_test = df_test.dropna(subset=['description']).reset_index(drop=True)
del df

In [32]:
df_train['clean_text'] = df_train["description"].apply(lambda row: text_preprocessing(row))
df_test['clean_text'] = df_test["description"].apply(lambda row: text_preprocessing(row))

In [33]:
log_reg = LogisticRegression(solver='newton-cg')
vectorizer = TfidfVectorizer(max_features=80000)

In [35]:
X_train_text = vectorizer.fit_transform(df_train['clean_text'].to_list())
X_test_text = vectorizer.transform(df_test['clean_text'].to_list())

In [36]:
pipe = Pipeline([('clf', log_reg)])

In [37]:
%%time
pipe.fit(X_train_text, df_train["clipTarget"].to_list())

CPU times: user 12min 2s, sys: 26min 27s, total: 38min 29s
Wall time: 6min 54s


Pipeline(steps=[('clf', LogisticRegression(solver='newton-cg'))])

In [26]:
print(roc_auc_score(df_test["clipTarget"].to_list(), pipe.predict_proba(X_test_text), multi_class='ovr')) # 30000
print(f1_score(df_test["clipTarget"].to_list(), pipe.predict(X_test_text), average='macro'))

0.8386465505010153
0.3070387112137004


In [31]:
print(roc_auc_score(df_test["clipTarget"].to_list(), pipe.predict_proba(X_test_text), multi_class='ovr')) # 50000
print(f1_score(df_test["clipTarget"].to_list(), pipe.predict(X_test_text), average='macro'))

0.839180563363717
0.3055044418060008


In [38]:
print(roc_auc_score(df_test["clipTarget"].to_list(), pipe.predict_proba(X_test_text), multi_class='ovr')) # 80000
print(f1_score(df_test["clipTarget"].to_list(), pipe.predict(X_test_text), average='macro'))

0.8466877204312302
0.3133577739036


In [43]:
print(roc_auc_score(df_test["clipTarget"].to_list(), pipe.predict_proba(X_test_text), multi_class='ovr')) # 100000
print(f1_score(df_test["clipTarget"].to_list(), pipe.predict(X_test_text), average='macro'))

0.8396065522460721
0.308082577156834


In [39]:
df = pd.read_csv(filename)


df = df[['date', 'clipTarget', 'asr']]
df_train = df.iloc[:324160, :]
df_train = df_train.dropna(subset=['asr']).reset_index(drop=True)


df_test = df.iloc[324160:, :]
df_test = df_test.dropna(subset=['asr']).reset_index(drop=True)
del df

df_train['clean_text'] = df_train["asr"].apply(lambda row: text_preprocessing(row))
df_test['clean_text'] = df_test["asr"].apply(lambda row: text_preprocessing(row))

In [40]:
log_reg = LogisticRegression(solver='newton-cg')
vectorizer = TfidfVectorizer(max_features=80000)

In [41]:
X_train_text = vectorizer.fit_transform(df_train['clean_text'].to_list())
X_test_text = vectorizer.transform(df_test['clean_text'].to_list())

In [42]:
pipe = Pipeline([('clf', log_reg)])

In [43]:
%%time
pipe.fit(X_train_text, df_train["clipTarget"].to_list())

CPU times: user 52.6 s, sys: 2min 21s, total: 3min 14s
Wall time: 13 s


Pipeline(steps=[('clf', LogisticRegression(solver='newton-cg'))])

In [44]:
# print(roc_auc_score(df_test["clipTarget"].to_list(), pipe.predict_proba(X_test_text), multi_class='ovr'))
print(f1_score(df_test["clipTarget"].to_list(), pipe.predict(X_test_text), average='macro'))

0.06689385781944765


In [52]:
df = pd.read_csv(filename)
df_train = df.iloc[:324160, :]
df_train = df_train.reset_index(drop=True)


df_test = df.iloc[324160:, :]
df_test = df_test.reset_index(drop=True)
del df

df_train["asr"] = df_train["asr"].apply(lambda row: row if isinstance(row, str) else "")
df_train["description"] = df_train["description"].apply(lambda row: row if isinstance(row, str) else "")
df_train["text"] = df_train["asr"] + df_train["description"]

columns = ["clipTarget", "text"]

df_train = df_train[df_train["text"] != ""].reset_index(drop=True)
df_train = df_train[columns].reset_index(drop=True)

df_train['clean_text'] = df_train["text"].apply(lambda row: text_preprocessing(row))

df_test["asr"] = df_test["asr"].apply(lambda row: row if isinstance(row, str) else "")
df_test["description"] = df_test["description"].apply(lambda row: row if isinstance(row, str) else "")
df_test["text"] = df_test["asr"] + df_test["description"]

df_test = df_test[df_test["text"] != ""].reset_index(drop=True)
df_test = df_test[columns].reset_index(drop=True)

df_test['clean_text'] = df_test["text"].apply(lambda row: text_preprocessing(row))

In [58]:
log_reg = LogisticRegression(solver='newton-cg')
vectorizer = TfidfVectorizer(max_features=80000)

In [54]:
X_train_text = vectorizer.fit_transform(df_train['clean_text'].to_list())
X_test_text = vectorizer.transform(df_test['clean_text'].to_list())

In [59]:
pipe = Pipeline([('clf', log_reg)])

In [60]:
%%time
pipe.fit(X_train_text, df_train["clipTarget"].to_list())

CPU times: user 12min 57s, sys: 27min 13s, total: 40min 11s
Wall time: 7min 27s


Pipeline(steps=[('clf', LogisticRegression(solver='newton-cg'))])

In [61]:
print(roc_auc_score(df_test["clipTarget"].to_list(), pipe.predict_proba(X_test_text), multi_class='ovr'))
print(f1_score(df_test["clipTarget"].to_list(), pipe.predict(X_test_text), average='macro'))

0.8471256501347992
0.315382266772482
